In [19]:
!pip install pyspark

In [17]:
from pyspark import SparkContext, SparkConf

import pyspark.sql as sql
from pyspark.sql import SparkSession
from pyspark.sql.types import DoubleType
from pyspark.sql.functions import udf, col, max, sum, countDistinct
from typing import NamedTuple
from datetime import datetime
from functools import reduce

In [22]:
app_name = "LR1_Apache_Spark"
conf = SparkConf().setAppName(app_name).setMaster('local[1]')
sc = SparkContext(conf=conf)
sc

<SparkContext master=local[1] appName=LR1_Apache_Spark>

In [24]:
def initStation(stations):
    class Station(NamedTuple):
        station_id: int
        name: str
        lat: float
        long: float
        dockcount: int
        landmark: str
        installation: str

    for station in stations:
        yield Station(
            station_id = int(station[0]),
            name = station[1],
            lat = float(station[2]),
            long = float(station[3]),
            dockcount = int(station[4]),
            landmark = station[5],
            installation = datetime.strptime(station[6], '%m/%d/%Y')
        )

def initTrip(trips):
    class Trip(NamedTuple):
        trip_id: int
        duration: int
        start_date: datetime
        start_station_name: str
        start_station_id: int
        end_date: datetime
        end_station_name: str
        end_station_id: int
        bike_id: int
        subscription_type: str
        zip_code: str

    for trip in trips:
        try:
            yield Trip(
             trip_id = int(trip[0]),
             duration = int(trip[1]),
             start_date = datetime.strptime(trip[2], '%m/%d/%Y %H:%M'),
             start_station_name = trip[3],
             start_station_id = int(trip[4]),
             end_date = datetime.strptime(trip[5], '%m/%d/%Y %H:%M'),
             end_station_name = trip[6],
             end_station_id = trip[7],
             bike_id = int(trip[8]),
             subscription_type = trip[9],
             zip_code = trip[10]
            )
        except:
            pass

In [41]:
trip_data = sc.textFile("trips.csv")
tripsHeader = trip_data.first()
trips = trip_data.filter(lambda row: row != tripsHeader).map(lambda row: row.split(",", -1))
stationData = sc.textFile("stations.csv")
stationsHeader = stationData.first()
stations = stationData.filter(lambda row: row != stationsHeader).map(lambda row: row.split(",", -1))

In [39]:
stationsIndexed = stations.keyBy(lambda station: station[0])

In [40]:
stationsIndexed.take(2)

[('2',
  ['2',
   'San Jose Diridon Caltrain Station',
   '37.329732',
   '-121.90178200000001',
   '27',
   'San Jose',
   '8/6/2013']),
 ('3',
  ['3',
   'San Jose Civic Center',
   '37.330698',
   '-121.888979',
   '15',
   'San Jose',
   '8/5/2013'])]

In [46]:
tripsByStartTerminals = trips.keyBy(lambda trip: trip[4])
tripsByEndTerminals = trips.keyBy(lambda trip: trip[7])

In [47]:
tripsByStartTerminals.take(2)

[('66',
  ['4576',
   '63',
   '',
   'South Van Ness at Market',
   '66',
   '8/29/2013 14:14',
   'South Van Ness at Market',
   '66',
   '520',
   'Subscriber',
   '94127']),
 ('10',
  ['4607',
   '',
   '8/29/2013 14:42',
   'San Jose City Hall',
   '10',
   '8/29/2013 14:43',
   'San Jose City Hall',
   '10',
   '661',
   'Subscriber',
   '95138'])]

In [34]:
tripsByEndTerminals.take(2)

[('66',
  ['4576',
   '63',
   '',
   'South Van Ness at Market',
   '66',
   '8/29/2013 14:14',
   'South Van Ness at Market',
   '66',
   '520',
   'Subscriber',
   '94127']),
 ('10',
  ['4607',
   '',
   '8/29/2013 14:42',
   'San Jose City Hall',
   '10',
   '8/29/2013 14:43',
   'San Jose City Hall',
   '10',
   '661',
   'Subscriber',
   '95138'])]

In [50]:
stations_mapped = stations.mapPartitions(initStation)

In [51]:
trips_mapped= trips.mapPartitions(initTrip)

##Задание 1. Найти велосипед с максимальным временем пробега

In [52]:
bike_max_mileage = trips_mapped.keyBy(lambda x: x.bike_id)
bike_duration = bike_max_mileage.mapValues(lambda x: x.duration).reduceByKey(lambda x1, x2: x1 + x2)
bike_duration_top = bike_duration.top(1, key=lambda x: x[1])[0][0]
bike_duration_top

print(f"Bike #{bike_duration_top} has max trips duration")

Bike #535 has max trips duration


## Задание 2. Найти наибольшее геодезическое расстояние между станциями


In [61]:
trips_stations = trips_mapped.filter(lambda trip: str(trip.start_station_id) != str(trip.end_station_id))\
                             .keyBy(lambda trip: (trip.start_station_id, trip.end_station_id))\
                             .mapValues(lambda trip: trip.duration)

In [62]:
query = trips_stations\
    .aggregateByKey(
        (0.0, 0.0),
        lambda acc, value: (acc[0] + value, acc[1] + 1),
        lambda acc1, acc2: (acc1[0] + acc2[0], acc1[1] + acc2[1]),)\
    .mapValues(lambda values: values[0] / values[1])

query.map(lambda x: x[::-1]).top(5)



[(229914.0, (26, '16')),
 (179212.5, (32, '63')),
 (169308.0, (80, '36')),
 (156461.03603603604, (66, '62')),
 (101207.5, (28, '2'))]

##Задание 3. Найти путь велосипеда с максимальным временем пробега через станции

In [63]:
bike_path = trips_mapped.filter(lambda x: x.bike_id == bike_duration_top)\
                        .sortBy(lambda x: x.start_date)\
                        .map(lambda x: (x.start_station_name, x.end_station_name))

bike_path.first()

('Post at Kearney', 'San Francisco Caltrain (Townsend at 4th)')

## Задаие 4. Найти количество велосипедов в системе.

In [64]:
count_bikes = trips_mapped.map(lambda x: x.bike_id).distinct().count()
count_bikes

700

## Задание 5. Найти пользователей потративших на поездки более 3 часов.

In [65]:
users = trips_mapped.filter(lambda x: x.duration > (3 * 60 * 60))\
                    .map(lambda x: x.zip_code)\
                    .filter(lambda x: x != "")\
                    .distinct()
users.take(10)

['58553',
 '94301',
 '94039',
 '94133',
 '93726',
 '94123',
 '4517',
 '29200',
 '45322',
 '94080']

In [66]:
sc.stop()